In [65]:
# required packages
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.model_selection import cross_val_score
import scipy as sp
import nltk
from combined import combined
from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix, f1_score

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#run combined.py on the essays
features = combined("LRS-training_set_rel3.csv")

adding given scores
adding essay set numbers
running org
running pos
running complexity


In [15]:
print(features[1])
#print(features)

{'complex_short': 0.0, 'pos_nums': 0, 'score': 4.0, 'complex_long': 1.0, 'pos_adjadv': 0.1282798833819242, 'pos_other': 0.4402332361516035, 'pos_verb': 0.20699708454810495, 'set': 1, 'organization': 'medium', 'complex_medium': 0.0, 'pos_noun': 0.22448979591836735, 'pos_pronoun': 0.0}


sklearn's SVM class takes an array X of size [n_samples, n_features] 
holding the training samples, and an array y of class labels (strings or integers), size [n_samples]
http://scikit-learn.org/stable/modules/svm.html
so we need to convert the data into this format
we also need to split by question set

In [16]:
#split dataset by question set
def split_by_set(features):
    """Returns a dictionary of sets mapped to a dictionary of essays in the set mapped to a dictionary of their features"""
    set = {}
    for k in features:
        #print(features[k])
        qset = features[k]['set']
        #print(k, " is in set ", qset)
        d = {k : features[k]}
        if qset in set:
            set[qset].update(d)
        else:
            set.update({qset : d})
    return set

In [22]:
sets = split_by_set(features)
print(sets[1][1])

{'complex_short': 0.0, 'pos_nums': 0, 'score': 4.0, 'complex_long': 1.0, 'pos_adjadv': 0.1282798833819242, 'pos_other': 0.4402332361516035, 'pos_verb': 0.20699708454810495, 'set': 1, 'organization': 'medium', 'complex_medium': 0.0, 'pos_noun': 0.22448979591836735, 'pos_pronoun': 0.0}


In [53]:
def get_set_arrays(setnum):
    num_essays = len(set[setnum])
    x = []
    y = []
    i = 0
    for e in set[setnum]:
        y.append(set[setnum][e]['score'])
        x.append([])
        for f in set[setnum][e]:
            if f != 'score' and f != 'set':
                if f == 'organization':
                    org_num = 0.0 #low organization
                    if set[setnum][e][f] == 'medium':
                        org_num = 0.5
                    elif set[setnum][e][f] == 'high':
                        org_num = 1.0
                    #append a number corresponding to the string feature
                    x[i].append(org_num)
                else:
                    x[i].append(set[setnum][e][f])
        i += 1
    return x, y

In [54]:
set1_data, set1_scores = get_set_arrays(1)
#print(set1_scores)
#print(set1_data)

BELOW: Running SVM with 10-fold cv, on separate question sets (except set 4), without Amer's vocab features

In [57]:
# now run SVM with 10-fold cross validation
#http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score
clf = svm.SVC()
cv_scores1 = cross_val_score(clf, set1_data, set1_scores, cv=10)
print("Set 1 Cross Validation Results:")
print(cv_scores1)

Set 1 Cross Validation Results:
[ 0.53888889  0.53888889  0.52513966  0.51955307  0.50837989  0.51955307
  0.52247191  0.53107345  0.53409091  0.51136364]


In [60]:
# Do the same for the rest of the essay sets
set2_data, set2_scores = get_set_arrays(2)
cv_scores2 = cross_val_score(clf, set2_data, set2_scores, cv=10)
print("Set 2 Cross Validation Results:")
print(cv_scores2)
set3_data, set3_scores = get_set_arrays(3)
cv_scores3 = cross_val_score(clf, set3_data, set3_scores, cv=10)
print("Set 3 Cross Validation Results:")
print(cv_scores3)
# set 4 has input that is breaking the code
#set4_data, set4_scores = get_set_arrays(4)
#cv_scores4 = cross_val_score(clf, set4_data, set4_scores, cv=10)
#print("Set 4 Cross Validation Results:")
#print(cv_scores4)
set5_data, set5_scores = get_set_arrays(5)
cv_scores5 = cross_val_score(clf, set5_data, set5_scores, cv=10)
print("Set 5 Cross Validation Results:")
print(cv_scores5)
set6_data, set6_scores = get_set_arrays(6)
cv_scores6 = cross_val_score(clf, set6_data, set6_scores, cv=10)
print("Set 6 Cross Validation Results:")
print(cv_scores6)
set7_data, set7_scores = get_set_arrays(7)
cv_scores7 = cross_val_score(clf, set7_data, set7_scores, cv=10)
print("Set 7 Cross Validation Results:")
print(cv_scores7)
set8_data, set8_scores = get_set_arrays(8)
cv_scores8 = cross_val_score(clf, set8_data, set8_scores, cv=10)
print("Set 8 Cross Validation Results:")
print(cv_scores8)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_split.py:579: Warning: The least populated class in y has only 7 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Set 2 Cross Validation Results:
[ 0.48087432  0.53551913  0.51912568  0.55248619  0.49444444  0.52513966
  0.52513966  0.48314607  0.52542373  0.47457627]
Set 3 Cross Validation Results:
[ 0.41954023  0.42774566  0.42196532  0.43352601  0.41618497  0.43604651
  0.43023256  0.44767442  0.39534884  0.43023256]
Set 5 Cross Validation Results:
[ 0.3989071   0.42857143  0.41208791  0.40659341  0.41758242  0.43093923
  0.42458101  0.41573034  0.40449438  0.42134831]
Set 6 Cross Validation Results:
[ 0.45901639  0.44262295  0.46703297  0.45        0.45        0.45810056
  0.46368715  0.4494382   0.45505618  0.45505618]


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_split.py:579: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Set 7 Cross Validation Results:
[ 0.24539877  0.25925926  0.25        0.26415094  0.2721519   0.27388535
  0.27922078  0.2745098   0.27631579  0.2781457 ]


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_split.py:579: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Set 8 Cross Validation Results:
[ 0.30120482  0.3125      0.32051282  0.33783784  0.34722222  0.33802817
  0.35294118  0.35820896  0.36363636  0.375     ]


The above was done with just one classifier, and as we tried with more sets it got progressively worse.
We probably need to do one classifier for each question set.

BELOW: Trying the above again with one classifier per question set

In [62]:
clf1 = svm.SVC()
cv_scores1b = cross_val_score(clf1, set1_data, set1_scores, cv=10)
print("Set 1 Cross Validation Results B:")
print(cv_scores1b)
clf2 = svm.SVC()
cv_scores2b = cross_val_score(clf2, set2_data, set2_scores, cv=10)
print("Set 2 Cross Validation Results B:")
print(cv_scores2b)
clf3 = svm.SVC()
cv_scores3b = cross_val_score(clf3, set3_data, set3_scores, cv=10)
print("Set 3 Cross Validation Results B:")
print(cv_scores3b)
# set 4 has input that is breaking the code
#clf4 = svm.SVC()
#cv_scores4b = cross_val_score(clf4, set4_data, set4_scores, cv=10)
#print("Set 4 Cross Validation Results B:")
#print(cv_scores4b)
clf5 = svm.SVC()
cv_scores5b = cross_val_score(clf5, set5_data, set5_scores, cv=10)
print("Set 5 Cross Validation Results B:")
print(cv_scores5b)
clf6 = svm.SVC()
cv_scores6b = cross_val_score(clf6, set6_data, set6_scores, cv=10)
print("Set 6 Cross Validation Results B:")
print(cv_scores6b)
clf7 = svm.SVC()
cv_scores7b = cross_val_score(clf7, set7_data, set7_scores, cv=10)
print("Set 7 Cross Validation Results B:")
print(cv_scores7b)
clf8 = svm.SVC()
cv_scores8b = cross_val_score(clf8, set8_data, set8_scores, cv=10)
print("Set 8 Cross Validation Results B:")
print(cv_scores8b)

Set 1 Cross Validation Results B:
[ 0.53888889  0.53888889  0.52513966  0.51955307  0.50837989  0.51955307
  0.52247191  0.53107345  0.53409091  0.51136364]


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_split.py:579: Warning: The least populated class in y has only 7 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Set 2 Cross Validation Results B:
[ 0.48087432  0.53551913  0.51912568  0.55248619  0.49444444  0.52513966
  0.52513966  0.48314607  0.52542373  0.47457627]
Set 3 Cross Validation Results B:
[ 0.41954023  0.42774566  0.42196532  0.43352601  0.41618497  0.43604651
  0.43023256  0.44767442  0.39534884  0.43023256]
Set 5 Cross Validation Results B:
[ 0.3989071   0.42857143  0.41208791  0.40659341  0.41758242  0.43093923
  0.42458101  0.41573034  0.40449438  0.42134831]
Set 6 Cross Validation Results B:
[ 0.45901639  0.44262295  0.46703297  0.45        0.45        0.45810056
  0.46368715  0.4494382   0.45505618  0.45505618]


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_split.py:579: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Set 7 Cross Validation Results B:
[ 0.24539877  0.25925926  0.25        0.26415094  0.2721519   0.27388535
  0.27922078  0.2745098   0.27631579  0.2781457 ]


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_split.py:579: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Set 8 Cross Validation Results B:
[ 0.30120482  0.3125      0.32051282  0.33783784  0.34722222  0.33802817
  0.35294118  0.35820896  0.36363636  0.375     ]


The above didn't seem to improve anything

BELOW: Try one classifier on all combined sets (except set 4), still no vocab features

In [64]:
all_sets_data = set1_data + set2_data + set3_data  + set5_data + set6_data + set7_data + set8_data
all_sets_scores = set1_scores + set2_scores + set3_scores + set5_scores + set6_scores + set7_scores + set8_scores
clf_all = svm.SVC()
cv_scores_all = cross_val_score(clf_all, all_sets_data, all_sets_scores, cv=10)
print("All sets Cross Validation Results:")
print(cv_scores_all)


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_split.py:579: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


All sets Cross Validation Results:
[ 0.26079295  0.2462489   0.23914969  0.26311111  0.31875     0.30473637
  0.31418312  0.25809353  0.22952295  0.21209386]


Try just running a different way for confusion matrix - DIDN"T WORK

In [75]:
clf_cm = svm.SVC()
clf_cm.fit(all_sets_data, all_sets_scores)
predictions = clf_cm.predict(all_sets_data)
cm = confusion_matrix(all_sets_scores, predictions)
#print(cm)

NOW TRY SHORT ANSWER, SAME THING
but using short_combined.py
We already have the features from this run on the github, just download and use those files

In [78]:
# Read all features into data frames
set1 = pd.read_csv("short_set1.csv")
set2 = pd.read_csv("short_set2.csv")
set3 = pd.read_csv("short_set3.csv")
set4 = pd.read_csv("short_set4.csv")
set5 = pd.read_csv("short_set5.csv")
set6 = pd.read_csv("short_set6.csv")
set7 = pd.read_csv("short_set7.csv")
set8 = pd.read_csv("short_set8.csv")
set9 = pd.read_csv("short_set9.csv")
set10 = pd.read_csv("short_set10.csv")
allsets = pd.read_csv("short_allsets.csv")

In [101]:
knowledge = pd.concat([set1,set2, set5, set6, set10])
english = pd.concat([set3, set4, set7, set8, set9])
# put sets into list for easy processing
all_data = [set1, set2, set3, set4, set5, set6, set7, set8, set9, set10, allsets, knowledge, english]

In [103]:
#print(knowledge)

In [104]:
# for each set
# please note that set 11 is actually the allsets data
# and set 12 is knowledge-based
# and set 13 is english-based
set_num = 1
for s in all_data:
    print("Splitting data for set ", set_num)
    #split into array of scores and array of other features
    scores = []
    features = []
    i = 0
    for row in s.itertuples(index=True, name='Pandas'):
        scores.append(getattr(row, "score"))
        features.append([])
        features[i].append(getattr(row, "org_score"))
        features[i].append(getattr(row, "pos_adjadv"))
        features[i].append(getattr(row, "pos_noun"))
        features[i].append(getattr(row, "pos_pronoun"))
        features[i].append(getattr(row, "pos_verb"))
        features[i].append(getattr(row, "pos_other"))
        features[i].append(getattr(row, "pos_nums"))
        features[i].append(getattr(row, "complex_short"))
        features[i].append(getattr(row, "complex_medium"))
        features[i].append(getattr(row, "complex_long"))
        features[i].append(getattr(row, "vocab_level"))

        i += 1

    #cross validation classifying for this set
    print("10-fold cross-validation results for set ", set_num)
    clf = svm.SVC()
    cv_scores = cross_val_score(clf, features, scores, cv=10)
    print(cv_scores)
    
    set_num += 1

Splitting data for set  1
10-fold cross-validation results for set  1
[ 0.35714286  0.35119048  0.38690476  0.33928571  0.37724551  0.36526946
  0.38323353  0.36526946  0.43712575  0.33333333]
Splitting data for set  2
10-fold cross-validation results for set  2
[ 0.37984496  0.43410853  0.34108527  0.34108527  0.35658915  0.35658915
  0.390625    0.36507937  0.384       0.36      ]
Splitting data for set  3
10-fold cross-validation results for set  3
[ 0.52356021  0.52910053  0.52380952  0.52380952  0.52910053  0.52910053
  0.52910053  0.52380952  0.52910053  0.52659574]
Splitting data for set  4
10-fold cross-validation results for set  4
[ 0.69142857  0.6         0.63218391  0.70114943  0.64942529  0.6091954
  0.65517241  0.71676301  0.68786127  0.63372093]
Splitting data for set  5
10-fold cross-validation results for set  5
[ 0.77472527  0.7679558   0.77777778  0.75        0.78212291  0.76536313
  0.77094972  0.77653631  0.78089888  0.76404494]
Splitting data for set  6
10-fold cr